In [1]:
#!/usr/bin/env python
import re
eval_abba = re.compile(r'([a-z])([a-z])(?!\1)\2\1')
eval_ssl = re.compile(r'([a-z])([a-z])(?!\2)\1')

def get_subnets(ip):
    supernets, hypernets = [], []
    for i in range(100):
        if '[' not in ip:
            supernets.append(ip)
            break
        supernets.append(ip[:ip.index('[')])
        hypernets.append(ip[ip.index('[')+1:ip.index(']', ip.index('['))])
        ip = ip[ip.index(']', ip.index('['))+1:]
    return supernets, hypernets

def eval_progressive_re(subnet, regex):
    pos, matches = 0, []
    while True:
        m = regex.search(subnet, pos)
        if m:
            pos = m.start()+1
            matches.append(m.group(0))
        else:
            return matches

def tls_capable(ip):
    supernets, hypernets = get_subnets(ip)
    if len([i for i in hypernets if eval_progressive_re(i, eval_abba)]):
        return False
    return len([i for i in supernets if eval_progressive_re(i, eval_abba)]) > 0

def ssl_capable(ip):
    supernets, hypernets = get_subnets(ip)
    supernet_matches = set([(i[1], i[2]) for supernet in supernets for i in eval_progressive_re(supernet, eval_ssl)])
    hypernet_matches = set([(i[2], i[1]) for hypernet in hypernets for i in eval_progressive_re(hypernet, eval_ssl)])
    return len(hypernet_matches & supernet_matches)


In [2]:
print eval_progressive_re('zazbz', eval_ssl)
print eval_progressive_re('bzb', eval_ssl)
assert ssl_capable('zazbz[bzb]cdb')

['zaz', 'zbz']
['bzb']


In [3]:
assert tls_capable('abba[mnop]qrst')
assert not tls_capable('abcd[bddb]xyyx')
assert not tls_capable('aaaa[qwer]tyui')
assert tls_capable('ioxxoj[asdfgh]zxcvbn')
assert tls_capable('ioxxoj[asdfgh]zxcvbnabba[mnop]qrst')
assert tls_capable('zxcvbn[asdfgh]qrst[mnop]ioxxoj')
assert not tls_capable('ioxxoj[asdfgh]zxcvbabcd[bddb]xyyx')
print "Solution 1 examples passed"
assert ssl_capable('aba[bab]xyz')
assert not ssl_capable('xyx[xyx]xyx')
assert ssl_capable('aaa[kek]eke')
assert ssl_capable('zazbz[bzb]cdb')
assert ssl_capable('zazabz[zaz]cdb')
print "Solution 2 examples passed"

Solution 1 examples passed
Solution 2 examples passed


In [4]:
total_ip_matching = 0
with open('Dec7.input', 'ra') as f:
    for input_str in f:
        if tls_capable(input_str):
            total_ip_matching += 1
print total_ip_matching

110


In [5]:
total_ip_matching = 0
with open('Dec7.input', 'ra') as f:
    for input_str in f:
        if ssl_capable(input_str):
            total_ip_matching += 1
print total_ip_matching

242
